<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px"> 
# Spark Machine Learning Model Loading
---

In [1]:
import pyspark as ps    
import warnings         
from pyspark.sql import SQLContext

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
try:
    # we try to create a SparkContext to work locally on all cpus available
    sc = ps.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    # give a warning if SparkContext already exists (for use inside pyspark)
    warnings.warn("SparkContext already exists in this scope")

Just created a SparkContext


In [3]:
evaluator = RegressionEvaluator(predictionCol='prediction',
                                labelCol='MEDV',
                                metricName='r2')

In [4]:
spark = ps.sql.SparkSession(sc)

spark_df = spark.read.csv(
    path='data/boston_housing.csv', 
    header=True,
    mode="DROPMALFORMED",
    inferSchema=True,
    enforceSchema=False
    )
(data_train, data_test) = spark_df.randomSplit([0.7, 0.3], seed=1)

In [5]:
best_model = PipelineModel.load('model-20190907-150636/')

predictions = best_model.transform(spark_df)
print(evaluator.evaluate(predictions))

predictions = best_model.transform(data_train)
print(evaluator.evaluate(predictions))

predictions = best_model.transform(data_test)
print(evaluator.evaluate(predictions))

0.7218117084843778
0.7310936297006648
0.6934885107911399
